<img style="float: center;" src='https://github.com/spacetelescope/jwst-pipeline-notebooks/raw/main/_static/stsci_header.png' alt="stsci_logo" width="900px"/> 

#  NIRISS Imaging Pipeline Notebook

**Authors**: S. LaMassa, R. Diaz<br>
**Last Updated**: September 3, 2024<br>
**Pipeline Version**: 1.14.1 (Build 10.2)

**Purpose**:

This notebook provides a framework for processing generic Near-Infrared
Imager and Slitless Spectrograph (NIRISS) Imaging data through all
three James Webb Space Telescope (JWST) pipeline stages.  Data is assumed
to be located in one observation folder according to paths set up below.
It should not be necessary to edit any cells other than in the 
[Configuration](#1.-Configuration) section unless modifying the standard 
pipeline processing options.

**Data**:
This example is set up to use an example dataset is from
[Program ID](https://www.stsci.edu/jwst/science-execution/program-information)
1475 (PI: Boyer, CoI: Volk) which is a sky flat calibration program. 
NIRCam is used as the primary instrument with NIRISS as a 
[coordinated parallel instrument](https://jwst-docs.stsci.edu/methods-and-roadmaps/jwst-parallel-observations/jwst-coordinated-parallels-roadmap).
The NIRISS imaging dataset uses a 17-step dither pattern.

Example input data to use will be downloaded automatically unless
disabled (i.e., to use local files instead).

**JWST pipeline version and CRDS context** This notebook was written for the
calibration pipeline version given above. It sets the CRDS context
to use the most recent version available in the JWST Calibration
Reference Data System (CRDS). If you use different pipeline versions or
CRDS context, please read the relevant release notes
([here for pipeline](https://github.com/spacetelescope/jwst),
[here for CRDS](https://jwst-crds.stsci.edu/)) for possibly relevant
changes.<BR>

**Updates**:
This notebook is regularly updated as improvements are made to the
pipeline. Find the most up to date version of this notebook at:
https://github.com/spacetelescope/jwst-pipeline-notebooks/

**Recent Changes**:<br>
January 24, 2024: original notebook released<br>
Septemer 3, 2024: Updated text to highlight that `IRAFStarFinder` is the default centroiding algorithm used in the `Image3` tweakreg step of the pipeline for NIRISS imaging, as of pipeline version 1.14.0 (build 10.2).<br>



## Table of Contents
1. [Configuration](#1.-Configuration) 
2. [Package Imports](#2.-Package-Imports)
3. [Demo Mode Setup (ignore if not using demo data)](#3.-Demo-Mode-Setup-(ignore-if-not-using-demo-data))
4. [Directory Setup](#4.-Directory-Setup)
3. [Detector 1 Pipelineg](#5.-Detector1-Pipeline)
4. [Image2 Pipeline](#6.-Image2-Pipeline)
5. [Image3 Pipeline](#7.-Image3-Pipeline)
6. [Visualize the data](#8.-Visualize-the-data)


## 1. Configuration
------------------
Set basic configuration for runing notebook. 

#### Install dependencies and parameters

To make sure that the pipeline version is compatabile with the steps
discussed below and the required dependencies and packages are installed,
 you can create a fresh conda environment and install the provided
`requirements.txt` file:
```
conda create -n niriss_imaging_pipeline python=3.11
conda activate niriss_imaging_pipeline
pip install -r requirements.txt
```

Set the basic parameters to use with this notebook. These will affect
what data is used, where data is located (if already in disk), and
pipeline modules run in this data. The list of parameters are:

* demo_mode
* directories with data
* pipeline modules

In [1]:
# Basic import necessary for configuration
import os

<div class="alert alert-block alert-warning">
Note that <code>demo_mode</code> must be set appropriately below.
</div>

Set <code>demo_mode = True </code> to run in demonstration mode. In this
mode this notebook will download example data from the Barbara A.
Mikulski Archive for Space Telescopes (MAST) and process it through the
pipeline. This will all happen in a local directory unless modified
in [Section 3](#3.-Demo-Mode-Setup-(ignore-if-not-using-demo-data))
below. 

Set <code>demo_mode = False</code> if you want to process your own data
that has already been downloaded and provide the location of the data.<br>

In [2]:
# Set parameters for demo_mode, channel, band, data mode directories, and 
# processing steps.

# -----------------------------Demo Mode---------------------------------
demo_mode = True

if demo_mode:
    print('Running in demonstration mode using online example data!')

# --------------------------User Mode Directories------------------------
# If demo_mode = False, look for user data in these paths
if not demo_mode:
    # Set directory paths for processing specific data; these will need
    # to be changed to your local directory setup (below are given as
    # examples)
    user_home_dir = os.path.expanduser('~')

    # Point to where science observation data are
    # Assumes uncalibrated data in sci_dir/uncal/ and results in stage1,
    # stage2, stage3 directories
    sci_dir = os.path.join(user_home_dir, 'FlightData/APT1475/data/Obs006/')

# --------------------------Set Processing Steps--------------------------
# Individual pipeline stages can be turned on/off here.  Note that a later
# stage won't be able to run unless data products have already been
# produced from the prior stage.

# Science processing
dodet1 = True  # calwebb_detector1
doimage2 = True  # calwebb_image2
doimage3 = True  # calwebb_image3

Running in demonstration mode using online example data!


### Set CRDS context and server
Before importing <code>CRDS</code> and <code>JWST</code> modules, we need
to configure our environment. This includes defining a CRDS cache
directory in which to keep the reference files that will be used by the
calibration pipeline.

If the root directory for the local CRDS cache directory has not been set
already, it will be set to create one in the home directory.

In [3]:
# ------------------------Set CRDS context and paths----------------------

# Set CRDS context (if overriding to use a specific version of reference
# files; leave commented out to use latest reference files by default)
#%env CRDS_CONTEXT  jwst_1254.pmap

# Check whether the local CRDS cache directory has been set.
# If not, set it to the user home directory
if (os.getenv('CRDS_PATH') is None):
    os.environ['CRDS_PATH'] = os.path.join(os.path.expanduser('~'), 'crds')
# Check whether the CRDS server URL has been set.  If not, set it.
if (os.getenv('CRDS_SERVER_URL') is None):
    os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'

# Echo CRDS path in use
print(f"CRDS local filepath: {os.environ['CRDS_PATH']}")
print(f"CRDS file server: {os.environ['CRDS_SERVER_URL']}")

CRDS local filepath: /Users/slamassa/crds
CRDS file server: https://jwst-crds.stsci.edu


## 2. Package Imports

In [4]:
# Use the entire available screen width for this notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [5]:
# Basic system utilities for interacting with files
# ----------------------General Imports------------------------------------
import glob
import time
from pathlib import Path
#import urllib.request

# Numpy for doing calculations
import numpy as np

# -----------------------Astroquery Imports--------------------------------
# ASCII files, and downloading demo files
from astroquery.mast import Observations

# For visualizing images
from jdaviz import Imviz

# Astropy routines for visualizing detected sources:
from astropy.table import Table
from astropy.coordinates import SkyCoord

# for JWST calibration pipeline
import jwst
import crds

from jwst.pipeline import Detector1Pipeline
from jwst.pipeline import Image2Pipeline
from jwst.pipeline import Image3Pipeline

# JWST pipeline utilities
from jwst import datamodels
from jwst.associations import asn_from_list  # Tools for creating association files
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base  # Definition of a Lvl3 association file

# Echo pipeline version and CRDS context in use
print(f"JWST Calibration Pipeline Version: {jwst.__version__}")
print(f"Using CRDS Context: {crds.get_context_name('jwst')}")

JWST Calibration Pipeline Version: 1.14.0
Using CRDS Context: jwst_1263.pmap


In [6]:
# Start a timer to keep track of runtime
time0 = time.perf_counter()

## 3. Demo Mode Setup (ignore if not using demo data)
------------------
If running in demonstration mode, set up the program information to
retrieve the uncalibrated data automatically from MAST using
[astroquery](https://astroquery.readthedocs.io/en/latest/mast/mast.html).
MAST allows for flexibility of searching by the proposal ID and the
observation ID instead of just filenames.<br>

For illustrative purposes, we focus on data taken through the NIRISS
[F150W filter](https://jwst-docs.stsci.edu/jwst-near-infrared-imager-and-slitless-spectrograph/niriss-instrumentation/niriss-filters)
and start with uncalibrated data products. The files are named
`jw01475006001_02201_000nn_nis_uncal.fits`, where *nn* refers to the
 dither step number which ranges from 01 - 17. 
 
More information about the JWST file naming conventions can be found at:
https://jwst-pipeline.readthedocs.io/en/latest/jwst/data_products/file_naming.html

In [7]:
# Set up the program information and paths for demo program
if demo_mode:
    print('Running in demonstration mode and will download example data from MAST!')
    program = '01475'
    sci_observtn = "006"
    visit = "001"
    data_dir = os.path.join('.', 'nis_im_demo_data')
    download_dir = data_dir
    sci_dir = os.path.join(data_dir, 'Obs' + sci_observtn)
    uncal_dir = os.path.join(sci_dir, 'uncal')

    # Ensure filepaths for input data exist
    if not os.path.exists(uncal_dir):
        os.makedirs(uncal_dir)
        
    # Create directory if it does not exist
    if not os.path.isdir(data_dir):
        os.mkdir(data_dir)


Running in demonstration mode and will download example data from MAST!


Identify list of science (SCI) uncalibrated files associated with visits.
<div class="alert alert-block alert-warning">
Selects only filter <i>f150w</i> data 
</div>

In [8]:
# Obtain a list of observation IDs for the specified demo program
if demo_mode:
    # Science data
    sci_obs_id_table = Observations.query_criteria(instrument_name=["NIRISS/IMAGE"],
                                                   provenance_name=["CALJWST"],  # Executed observations
                                                   filters=['F150W'],  # Data for Specific Filter
                                                   obs_id=['jw' + program + '-o' + sci_observtn + '*']
                                                   )

In [9]:
# Turn the list of visits into a list of uncalibrated data files
if demo_mode:
    # Define types of files to select
    file_dict = {'uncal': {'product_type': 'SCIENCE',
                           'productSubGroupDescription': 'UNCAL',
                           'calib_level': [1]}}

    # Science files
    sci_files_to_download = []
    # Loop over visits identifying uncalibrated files that are associated
    # with them
    for exposure in (sci_obs_id_table):
        products = Observations.get_product_list(exposure)
        for filetype, query_dict in file_dict.items():
            filtered_products = Observations.filter_products(products, productType=query_dict['product_type'],
                                                             productSubGroupDescription=query_dict['productSubGroupDescription'],
                                                             calib_level=query_dict['calib_level'])
            sci_files_to_download.extend(filtered_products['dataURI'])
 
    sci_files_to_download = sorted(sci_files_to_download)
    print(f"Science files selected for downloading: {len(sci_files_to_download)}")

Science files selected for downloading: 17


Download all the uncal files and place them into the appropriate
directories.

<div class="alert alert-block alert-warning">
Warning: If this notebook is halted during this step the downloaded file
may be incomplete, and cause crashes later on!
</div>

In [10]:
if demo_mode:
    for filename in sci_files_to_download:
        sci_manifest = Observations.download_file(filename,
                                                  local_path=os.path.join(uncal_dir, Path(filename).name))

2024-08-26 15:38:20,755 - stpipe - INFO - Found cached file ./nis_im_demo_data/Obs006/uncal/jw01475006001_02201_00001_nis_uncal.fits with expected size 134271360.


INFO: Found cached file ./nis_im_demo_data/Obs006/uncal/jw01475006001_02201_00001_nis_uncal.fits with expected size 134271360. [astroquery.query]


2024-08-26 15:38:20,896 - stpipe - INFO - Found cached file ./nis_im_demo_data/Obs006/uncal/jw01475006001_02201_00002_nis_uncal.fits with expected size 134271360.


INFO: Found cached file ./nis_im_demo_data/Obs006/uncal/jw01475006001_02201_00002_nis_uncal.fits with expected size 134271360. [astroquery.query]


2024-08-26 15:38:21,035 - stpipe - INFO - Found cached file ./nis_im_demo_data/Obs006/uncal/jw01475006001_02201_00003_nis_uncal.fits with expected size 134271360.


...

INFO: Found cached file ./nis_im_demo_data/Obs006/uncal/jw01475006001_02201_00015_nis_uncal.fits with expected size 134271360. [astroquery.query]


2024-08-26 15:38:22,817 - stpipe - INFO - Found cached file ./nis_im_demo_data/Obs006/uncal/jw01475006001_02201_00016_nis_uncal.fits with expected size 134271360.


INFO: Found cached file ./nis_im_demo_data/Obs006/uncal/jw01475006001_02201_00016_nis_uncal.fits with expected size 134271360. [astroquery.query]


2024-08-26 15:38:22,955 - stpipe - INFO - Found cached file ./nis_im_demo_data/Obs006/uncal/jw01475006001_02201_00017_nis_uncal.fits with expected size 134271360.


INFO: Found cached file ./nis_im_demo_data/Obs006/uncal/jw01475006001_02201_00017_nis_uncal.fits with expected size 134271360. [astroquery.query]


## 4. Directory Setup
------------------
Set up detailed paths to input/output stages here.

In [11]:
# Define output subdirectories to keep science data products organized
uncal_dir = os.path.join(sci_dir, 'uncal')  # Uncalibrated pipeline inputs should be here
det1_dir = os.path.join(sci_dir, 'stage1')  # calwebb_detector1 pipeline outputs will go here
image2_dir = os.path.join(sci_dir, 'stage2')  # calwebb_spec2 pipeline outputs will go here
image3_dir = os.path.join(sci_dir, 'stage3')  # calwebb_spec3 pipeline outputs will go here

# We need to check that the desired output directories exist, and if not
# create them
if not os.path.exists(det1_dir):
    os.makedirs(det1_dir)
if not os.path.exists(image2_dir):
    os.makedirs(image2_dir)
if not os.path.exists(image3_dir):
    os.makedirs(image3_dir)

Look at the first file to determine exposure parameters and practice using
JWST datamodels¶

In [12]:
uncal_files = sorted(glob.glob(os.path.join(uncal_dir, '*_uncal.fits')))

# print file name
print(uncal_files[0])

# Open file as JWST datamodel
examine = datamodels.open(uncal_files[0])

# Print out exposure info
print(f"Instrument: {examine.meta.instrument.name}")
print(f"Filter: {examine.meta.instrument.filter}")
print(f"Pupil: {examine.meta.instrument.pupil}")
print(f"Number of integrations: {examine.meta.exposure.nints}")
print(f"Number of groups: {examine.meta.exposure.ngroups}")
print(f"Readout pattern: {examine.meta.exposure.readpatt}")
print(f"Dither position number: {examine.meta.dither.position_number}")
print("\n")

./nis_im_demo_data/Obs006/uncal/jw01475006001_02201_00001_nis_uncal.fits
Instrument: NIRISS
Filter: CLEAR
Pupil: F150W
Number of integrations: 1
Number of groups: 16
Readout pattern: NIS
Dither position number: 1




From the above, we confirm that the data file is for the NIRISS instrument
using the `F150W` filter in the [Pupil Wheel](https://jwst-docs.stsci.edu/jwst-near-infrared-imager-and-slitless-spectrograph/niriss-instrumentation/niriss-pupil-and-filter-wheels)
crossed with the `CLEAR` filter in the Filter Wheel. This observation uses
the [`NIS` readout pattern](https://jwst-docs.stsci.edu/jwst-near-infrared-imager-and-slitless-spectrograph/niriss-instrumentation/niriss-detector-overview/niriss-detector-readout-patterns),
16 groups per integration, and 1 integration per exposure. This data file
is the 1st dither position in this exposure sequence. For more information
about how JWST exposures are defined by up-the-ramp sampling, see the
[Understanding Exposure Times JDox article](https://jwst-docs.stsci.edu/understanding-exposure-times).

This metadata will be the same for all exposures in this observation other
than the dither position number.

In [13]:
# Print out the time benchmark
time1 = time.perf_counter()
print(f"Runtime so far: {time1 - time0:0.0f} seconds")

Runtime so far: 30 seconds


## 5. Detector1 Pipeline
Run the datasets through the
[Detector1](https://jwst-docs.stsci.edu/jwst-science-calibration-pipeline-overview/stages-of-jwst-data-processing/calwebb_detector1)
stage of the pipelineto apply detector level calibrations and create a
countrate data product where slopes are fitted to the integration ramps.
These `*_rate.fits` products are 2D (nrows x ncols), averaged over all
integrations. 3D countrate data products (`*_rateints.fits`) are also
created (nintegrations x nrows x ncols) which have the fitted ramp slopes
for each integration.

By default, all steps in the Detector1 stage of the pipeline are run for
NIRISS except: the `ipc` correction step and the `gain_scale` step. Note
that while the [`persistence` step](https://jwst-pipeline.readthedocs.io/en/latest/jwst/persistence/description.html)
is set to run by default, this step does not automatically correct the
science data for persistence. The `persistence` step creates a
`*_trapsfilled.fits` file which is a model that records the number
of traps filled at each pixel at the end of an exposure. This file would be 
used as an input to the `persistence` step, via the `input_trapsfilled`
argument, to correct a science exposure for persistence. Since persistence
is not well calibrated for NIRISS, we do not perform a persistence
correction and thus turn off this step to speed up calibration and to not
create files that will not be used in the subsequent analysis. This step
can be turned off when running the pipeline in Python by doing:
```
rate_result = Detector1Pipeline.call(uncal,steps={'persistence': {'skip': True}})
```
or as indicated in the cell bellow using a dictionary.

The [charge_migration step](https://jwst-pipeline.readthedocs.io/en/latest/jwst/charge_migration/index.html#charge-migration-step)
is particularly important for NIRISS images to mitigate apparent flux loss
in resampled images due to the spilling of charge from a central pixel into
its neighboring pixels (see [Goudfrooij et al. 2023](https://ui.adsabs.harvard.edu/abs/2023arXiv231116301G/abstract)
for details). Charge migration occurs when the accumulated charge in a
central pixel exceeds a certain signal limit, which is ~25,000 ADU. This
step is turned on by default for NIRISS imaging mode when using CRDS
contexts of `jwst_1159.pmap` or later. Different signal limits for each filter are provided by the 
[pars-chargemigrationstep parameter files](https://jwst-crds.stsci.edu). 
Users can specify a different signal limit by running this step with the
`signal_threshold` flag and entering another signal limit in units of ADU.

As of CRDS context `jwst_1155.pmap` and later, the 
[`jump` step](https://jwst-pipeline.readthedocs.io/en/latest/api/jwst.jump.JumpStep.html)
of the `DETECTOR1` stage of the pipeline will remove residuals associated
 with [snowballs](https://jwst-docs.stsci.edu/known-issues-with-jwst-data/shower-and-snowball-artifacts)
 for the NIRISS imaging mode. The default parameters for this correction,
 where `expand_large_events` set to `True` turns on the snowball halo
 removal algorithm, are specified in the `pars-jumpstep` parameter
 reference files. Users may wish to alter parameters to optimize removal of
 snowball residuals. Available parameters are discussed in the
 [Detection and Flagging of Showers and Snowballs in JWST Technical Report (Regan 2023)](https://www.stsci.edu/files/live/sites/www/files/home/jwst/documentation/technical-documents/_documents/JWST-STScI-008545.pdf).

In [14]:
# Set up a dictionary to define how the Detector1 pipeline should be configured

# Boilerplate dictionary setup
det1dict = {}
det1dict['group_scale'], det1dict['dq_init'], det1dict['saturation'] = {}, {}, {}
det1dict['ipc'], det1dict['superbias'], det1dict['refpix'] = {}, {}, {}
det1dict['linearity'], det1dict['persistence'], det1dict['dark_current'], = {}, {}, {}
det1dict['charge_migration'], det1dict['jump'], det1dict['ramp_fit'] = {}, {}, {}
det1dict['gain_scale'] = {}

# Overrides for whether or not certain steps should be skipped
# skipping the persistence step
det1dict['persistence']['skip'] = True
#det1dict['jump']['find_showers'] = False # Turn off detection of cosmic ray showers

# Overrides for various reference files
# Files should be in the base local directory or provide full path
#det1dict['dq_init']['override_mask'] = 'myfile.fits' # Bad pixel mask
#det1dict['saturation']['override_saturation'] = 'myfile.fits' # Saturation
#det1dict['reset']['override_reset'] = 'myfile.fits' # Reset
#det1dict['linearity']['override_linearity'] = 'myfile.fits' # Linearity
#det1dict['rscd']['override_rscd'] = 'myfile.fits' # RSCD
#det1dict['dark_current']['override_dark'] = 'myfile.fits' # Dark current subtraction
#det1dict['jump']['override_gain'] = 'myfile.fits' # Gain used by jump step
#det1dict['ramp_fit']['override_gain'] = 'myfile.fits' # Gain used by ramp fitting step
#det1dict['jump']['override_readnoise'] = 'myfile.fits' # Read noise used by jump step
#det1dict['ramp_fit']['override_readnoise'] = 'myfile.fits' # Read noise used by ramp fitting step

# Turn on multi-core processing (off by default).  Choose what fraction of cores to use (quarter, half, or all)
det1dict['jump']['maximum_cores'] = 'half'

# Alter parameters to optimize removal of snowball residuals (example)
#det1dict['jump']['expand_large_events'] = True
#det1dict['charge_migration']['signal_threshold']= X


In [15]:
# Run Detector1 stage of pipeline, specifying:
# output directory to save *_rate.fits files
# save_results flag set to True so the rate files are saved

if dodet1:
    for uncal in uncal_files:
        rate_result = Detector1Pipeline.call(uncal, output_dir=det1_dir, steps=det1dict, save_results=True,)
else:
    print('Skipping Detector1 processing')

2024-08-26 15:38:50,191 - stpipe - INFO - PARS-CHARGEMIGRATIONSTEP parameters found: /Users/slamassa/crds/references/jwst/niriss/jwst_niriss_pars-chargemigrationstep_0018.asdf
2024-08-26 15:38:50,198 - stpipe - INFO - PARS-JUMPSTEP parameters found: /Users/slamassa/crds/references/jwst/niriss/jwst_niriss_pars-jumpstep_0087.asdf
2024-08-26 15:38:50,205 - stpipe - INFO - PARS-DETECTOR1PIPELINE parameters found: /Users/slamassa/crds/references/jwst/niriss/jwst_niriss_pars-detector1pipeline_0001.asdf
2024-08-26 15:38:50,216 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2024-08-26 15:38:50,216 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2024-08-26 15:38:50,217 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2024-08-26 15:38:50,218 - stpipe.Detector1Pipeline.emicorr - INFO - EmiCorrStep instance created.
2024-08-26 15:38:50,218 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created

In [16]:
# Print out the time benchmark
time1 = time.perf_counter()
print(f"Runtime for Detector1: {time1 - time0:0.0f} seconds")

Runtime for Detector1: 2804 seconds


### Exploring the data

Identify `*_rate.fits` files and verify which pipeline steps were run and
which calibration reference files were applied.

The header contains information about which calibration steps were
completed and skipped and which reference files were used to process the
data.

In [17]:
# find rate files
rate_files = sorted(glob.glob(os.path.join(det1_dir, '*_rate.fits')))

# Read in file as datamodel
rate_f = datamodels.open(rate_files[0])

# Check which steps were run
rate_f.meta.cal_step.instance

{'charge_migration': 'COMPLETE',
 'dark_sub': 'COMPLETE',
 'dq_init': 'COMPLETE',
 'gain_scale': 'SKIPPED',
 'group_scale': 'SKIPPED',
 'ipc': 'SKIPPED',
 'jump': 'COMPLETE',
 'linearity': 'COMPLETE',
 'persistence': 'SKIPPED',
 'ramp_fit': 'COMPLETE',
 'refpix': 'COMPLETE',
 'saturation': 'COMPLETE',
 'superbias': 'COMPLETE'}

Check which reference files were used to calibrate the dataset:

In [18]:
rate_f.meta.ref_file.instance

{'crds': {'context_used': 'jwst_1263.pmap', 'sw_version': '11.18.1'},
 'dark': {'name': 'crds://jwst_niriss_dark_0169.fits'},
 'gain': {'name': 'crds://jwst_niriss_gain_0006.fits'},
 'linearity': {'name': 'crds://jwst_niriss_linearity_0017.fits'},
 'mask': {'name': 'crds://jwst_niriss_mask_0017.fits'},
 'readnoise': {'name': 'crds://jwst_niriss_readnoise_0005.fits'},
 'saturation': {'name': 'crds://jwst_niriss_saturation_0015.fits'},
 'superbias': {'name': 'crds://jwst_niriss_superbias_0183.fits'}}

## 6. Image2 Pipeline 

In the [Image2 stage of the pipeline](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_image2.html),
calibrated unrectified data products are created (`*_cal.fits` or
`*_calints.fits` files, depending on whether the input files are
`*_rate.fits` or `*_rateints.fits`). 

In this pipeline processing stage, the [world coordinate system (WCS)](https://jwst-pipeline.readthedocs.io/en/latest/jwst/assign_wcs/index.html#assign-wcs-step)
is assigned, the data are [flat fielded](https://jwst-pipeline.readthedocs.io/en/latest/jwst/flatfield/index.html#flatfield-step),
and a [photometric calibration](https://jwst-pipeline.readthedocs.io/en/latest/jwst/photom/index.html#photom-step)
is applied to convert from units of countrate (ADU/s) to surface brightness (MJy/sr).

By default, the [background subtraction step](https://jwst-pipeline.readthedocs.io/en/latest/jwst/background_step/index.html#background-step)
and the [resampling step](https://jwst-pipeline.readthedocs.io/en/latest/jwst/resample/index.html#resample-step)
are turned off for NIRISS at this stage of the pipeline. The background
subtraction is turned off since there is no background template for the
imaging mode and the local background is removed during the background
correction for photometric measurements around individual sources. The
resampling step occurs during the `Image3` stage by default. While the
resampling step can be turned on during the `Image2` stage to, e.g., 
generate a source catalog for each image, the data quality from the
`Image3` stage will be better since the bad pixels, which adversely affect
both the centroids and photometry in individual images, will be mostly
removed.

In [19]:
time_image2 = time.perf_counter()

In [20]:
# Set up a dictionary to define how the Image2 pipeline should be configured.

# Boilerplate dictionary setup
image2dict = {}
image2dict['assign_wcs'], image2dict['flat_field'] = {}, {}
image2dict['photom'], image2dict['resample'] = {}, {}

# Overrides for whether or not certain steps should be skipped (example)
#image2dict['resample']['skip'] = False

# Overrides for various reference files
# Files should be in the base local directory or provide full path
#image2dict['assign_wcs']['override_distortion'] = 'myfile.asdf' # Spatial distortion (ASDF file)
#image2dict['assign_wcs']['override_filteroffset'] = 'myfile.asdf' # Imager filter offsets (ASDF file)
#image2dict['assign_wcs']['override_specwcs'] = 'myfile.asdf' # Spectral distortion (ASDF file)
#image2dict['assign_wcs']['override_wavelengthrange'] = 'myfile.asdf' # Wavelength channel mapping (ASDF file)
#image2dict['flat_field']['override_flat'] = 'myfile.fits' # Pixel flatfield
#image2dict['photom']['override_photom'] = 'myfile.fits' # Photometric calibration array

Find and sort all of the input files, ensuring use of absolute paths

In [21]:
sstring = os.path.join(det1_dir, 'jw*rate.fits')  # Use files from the detector1 output folder
rate_files = sorted(glob.glob(sstring))
for ii in range(0, len(rate_files)):
    rate_files[ii] = os.path.abspath(rate_files[ii])
rate_files = np.array(rate_files)
print(f"Found  {str(len(rate_files))} science files")

Found  17 science files


In [22]:
# Run Image2 stage of pipeline, specifying:
# output directory to save *_cal.fits files
# save_results flag set to True so the rate files are saved

if doimage2:
    for rate in rate_files:
        cal_result = Image2Pipeline.call(rate, output_dir=image2_dir, steps=image2dict, save_results=True)
else:
    print("Skipping Image2 processing.")


2024-08-26 16:25:21,590 - stpipe - INFO - PARS-RESAMPLESTEP parameters found: /Users/slamassa/crds/references/jwst/niriss/jwst_niriss_pars-resamplestep_0001.asdf
2024-08-26 16:25:21,597 - stpipe - INFO - PARS-IMAGE2PIPELINE parameters found: /Users/slamassa/crds/references/jwst/niriss/jwst_niriss_pars-image2pipeline_0002.asdf
2024-08-26 16:25:21,605 - stpipe.Image2Pipeline - INFO - Image2Pipeline instance created.
2024-08-26 16:25:21,606 - stpipe.Image2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2024-08-26 16:25:21,607 - stpipe.Image2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2024-08-26 16:25:21,607 - stpipe.Image2Pipeline.flat_field - INFO - FlatFieldStep instance created.
2024-08-26 16:25:21,608 - stpipe.Image2Pipeline.photom - INFO - PhotomStep instance created.
2024-08-26 16:25:21,609 - stpipe.Image2Pipeline.resample - INFO - ResampleStep instance created.
2024-08-26 16:25:21,940 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline running with

In [23]:
# Print out the time benchmark
time1 = time.perf_counter()
print(f"Runtime so far: {time1 - time0:0.0f} seconds")
print(f"Runtime for Image2: {time1 - time_image2:0.0f} seconds")

Runtime so far: 3979 seconds
Runtime for Image2: 1157 seconds


### Verify which pipeline steps were run

In [24]:
# Identify *_cal.fits files
cal_files = sorted(glob.glob(os.path.join(image2_dir, '*_cal.fits')))

cal_f = datamodels.open(cal_files[0])

# Check which steps were run:
cal_f.meta.cal_step.instance

{'assign_wcs': 'COMPLETE',
 'charge_migration': 'COMPLETE',
 'dark_sub': 'COMPLETE',
 'dq_init': 'COMPLETE',
 'flat_field': 'COMPLETE',
 'gain_scale': 'SKIPPED',
 'group_scale': 'SKIPPED',
 'ipc': 'SKIPPED',
 'jump': 'COMPLETE',
 'linearity': 'COMPLETE',
 'persistence': 'SKIPPED',
 'photom': 'COMPLETE',
 'ramp_fit': 'COMPLETE',
 'refpix': 'COMPLETE',
 'resample': 'SKIPPED',
 'saturation': 'COMPLETE',
 'superbias': 'COMPLETE'}

Check which reference files were used to calibrate the dataset:

In [25]:
cal_f.meta.ref_file.instance

{'area': {'name': 'crds://jwst_niriss_area_0021.fits'},
 'camera': {'name': 'N/A'},
 'collimator': {'name': 'N/A'},
 'crds': {'context_used': 'jwst_1263.pmap', 'sw_version': '11.18.1'},
 'dark': {'name': 'crds://jwst_niriss_dark_0169.fits'},
 'dflat': {'name': 'N/A'},
 'disperser': {'name': 'N/A'},
 'distortion': {'name': 'crds://jwst_niriss_distortion_0047.asdf'},
 'fflat': {'name': 'N/A'},
 'filteroffset': {'name': 'crds://jwst_niriss_filteroffset_0006.asdf'},
 'flat': {'name': 'crds://jwst_niriss_flat_0282.fits'},
 'fore': {'name': 'N/A'},
 'fpa': {'name': 'N/A'},
 'gain': {'name': 'crds://jwst_niriss_gain_0006.fits'},
 'ifufore': {'name': 'N/A'},
 'ifupost': {'name': 'N/A'},
 'ifuslicer': {'name': 'N/A'},
 'linearity': {'name': 'crds://jwst_niriss_linearity_0017.fits'},
 'mask': {'name': 'crds://jwst_niriss_mask_0017.fits'},
 'msa': {'name': 'N/A'},
 'ote': {'name': 'N/A'},
 'photom': {'name': 'crds://jwst_niriss_photom_0043.fits'},
 'readnoise': {'name': 'crds://jwst_niriss_readno

## 7. Image3 Pipeline

In the [Image3 stage of the pipeline](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_image3.html), the individual `*_cal.fits` files for each of the dither positions are combined to one single distortion corrected image. First, an [Association](https://jwst-pipeline.readthedocs.io/en/latest/jwst/associations/overview.html) needs to be created to inform the pipeline that these individual exposures are linked together. 

By default, the `Image3` stage of the pipeline performs the following steps on NIRISS data: 
- [tweakreg](https://jwst-pipeline.readthedocs.io/en/latest/jwst/tweakreg/index.html#tweakreg-step) - creates source catalogs of pointlike sources for each input image. The source catalog for each input image is compared to each other to derive coordinate transforms to align the images relative to each other.
   - As of CRDS context `jwst_1156.pmap` and later, the `pars-tweakreg` parameter reference file for NIRISS performs an absolute astrometric correction to GAIA data release 3 by default (i.e., the `abs_refcat` parameter is set to `GAIADR3`). Though this default correction generally improves results compared with not doing this alignment, it could potentially result in poor performance in crowded or sparse fields, so users are encouraged to check astrometric accuracy and revisit this step if necessary.
   - As of pipeline version 1.14.0, the default source finding algorithm for NIRISS is `IRAFStarFinder` which testing shows returns good accuracy for undersampled NIRISS PSFs at short wavelengths [(Goudfrooij 2022)](https://www.stsci.edu/files/live/sites/www/files/home/jwst/documentation/technical-documents/_documents/JWST-STScI-008116.pdf). 
- [skymatch](https://jwst-pipeline.readthedocs.io/en/latest/jwst/skymatch/index.html#skymatch-step) - measures the background level from the sky to use as input into the subsequent `outlier detection` and `resample` steps.
- [outlier detection](https://jwst-pipeline.readthedocs.io/en/latest/jwst/outlier_detection/index.html#outlier-detection-step) - flags any remaining cosmic rays, bad pixels, or other artifacts not already flagged during the `DETECTOR1` stage of the pipeline, using all input images to create a median image so that outliers in individual images can be identified.
- [resample](https://jwst-pipeline.readthedocs.io/en/latest/jwst/resample/index.html#resample-step) - resamples each input image based on its WCS and distortion information and creates a single undistorted image.
- [source catalog](https://jwst-pipeline.readthedocs.io/en/latest/jwst/source_catalog/index.html#source-catalog-step) - creates a catalog of detected sources along with measured photometries and morphologies (i.e., point-like vs extended). Useful for quicklooks, but optimization is likely needed for specific science cases, which is an on-going investigation for the NIRISS team. Users may wish to experiment with changing the `snr_threshold` and `deblend` options. Modifications to the following parameters will not significantly improve data quality and it is advised to keep them at their default values: `aperture_ee1`, `aperture_ee2`, `aperture_ee3`, `ci1_star_threshold`, `ci2_star_threshold`.


In [26]:
time_image3 = time.perf_counter()

In [27]:
# Set up a dictionary to define how the Image3 pipeline should be configured

# Boilerplate dictionary setup
image3dict = {}
image3dict['assign_mtwcs'], image3dict['tweakreg'], image3dict['skymatch'] = {}, {}, {}
image3dict['outlier_detection'], image3dict['resample'], image3dict['source_catalog'] = {}, {}, {}

# Overrides for whether or not certain steps should be skipped (example)
#image3dict['outlier_detection']['skip'] = True

# Overrides for various reference files
# Files should be in the base local directory or provide full path
#image3dict['source_catalog']['override_apcorr'] = 'myfile.fits'  # Aperture correction parameters
#image3dict['source_catalog']['override_abvegaoffset'] = 'myfile.asdf' # Data to convert from AB to Vega magnitudes (ASDF file)

Find and sort all of the input files, ensuring use of absolute paths

In [28]:
# Science Files need the cal.fits files
sstring = os.path.join(image2_dir, 'jw*cal.fits')
cal_files = sorted(glob.glob(sstring))
for ii in range(0, len(cal_files)):
    cal_files[ii] = os.path.abspath(cal_files[ii])
calfiles = np.array(cal_files)

print(f'Found {str(len(cal_files))} science files to process')

Found 17 science files to process


### Create Association File

An association file lists the exposures to calibrated together in `Stage 3` of the pipeline. Note that an association file is available for download from MAST, with a filename of `*_asn.json`. Here we show how to create an association file to point to the data products created when processing data through the pipeline. Note that the output products will have a rootname that is specified by the `product_name` in the association file. For this tutorial, the rootname of the output products will be `image3_association`.

In [29]:
# Create a Level 3 Association
if doimage3:
    associations = asn_from_list.asn_from_list(cal_files,
                                               rule=DMS_Level3_Base,
                                               product_name='image3_association')
    
    associations.data['asn_type'] = 'image3'
    associations.data['program'] = program
    
    # Format association as .json file
    asn_filename, serialized = associations.dump(format="json")

    # Write out association file
    association_im3 = os.path.join(sci_dir, asn_filename) 
    with open(association_im3, "w") as fd:
        fd.write(serialized)

2024-08-26 16:45:00,556 - stpipe - WARNING - /Users/slamassa/opt/miniconda3/envs/niriss_imaging_pipeline/lib/python3.11/site-packages/jwst/associations/association.py:215: UserWarning: 'expname' contains path, but should only be a filename.  All input files should be in a single directory, so no path is needed.
2024-08-26 16:45:00,557 - stpipe - WARNING -   warnings.warn(err_str, UserWarning)
2024-08-26 16:45:00,557 - stpipe - WARNING - 


### Run Image3 stage of the pipeline

Given the grouped exposures in the association file, the `Image3` stage of the pipeline will produce:
- a `*_cr.fits` file produced by the `outlier_detection` step, where the `DQ` array marks the pixels flagged as outliers.
- a final combined, rectified image with name `*_i2d.fits`,
- a source catalog with name `*_cat.ecsv`,
- a segmentation map file (`*_segm.fits`) which has integer values at the pixel locations where a source is detected where the pixel values match the source ID number in the catalog.

In [30]:
# Run Stage 3

if doimage3:
    i2d_result = Image3Pipeline.call(association_im3, output_dir=image3_dir, steps=image3dict, save_results=True)
else:
    print('Skipping Spec3 processing')

2024-08-26 16:45:00,828 - stpipe - INFO - PARS-TWEAKREGSTEP parameters found: /Users/slamassa/crds/references/jwst/niriss/jwst_niriss_pars-tweakregstep_0079.asdf
2024-08-26 16:45:00,835 - stpipe - INFO - PARS-OUTLIERDETECTIONSTEP parameters found: /Users/slamassa/crds/references/jwst/niriss/jwst_niriss_pars-outlierdetectionstep_0010.asdf
2024-08-26 16:45:00,844 - stpipe - INFO - PARS-RESAMPLESTEP parameters found: /Users/slamassa/crds/references/jwst/niriss/jwst_niriss_pars-resamplestep_0001.asdf
2024-08-26 16:45:00,849 - stpipe - INFO - PARS-SOURCECATALOGSTEP parameters found: /Users/slamassa/crds/references/jwst/niriss/jwst_niriss_pars-sourcecatalogstep_0011.asdf
2024-08-26 16:45:00,858 - stpipe.Image3Pipeline - INFO - Image3Pipeline instance created.
2024-08-26 16:45:00,859 - stpipe.Image3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2024-08-26 16:45:00,860 - stpipe.Image3Pipeline.tweakreg - INFO - TweakRegStep instance created.
2024-08-26 16:45:00,861 - stpipe.Im

In [31]:
# Print out the time benchmark
time1 = time.perf_counter()
print(f"Runtime so far: {time1 - time0:0.0f} seconds")
print(f"Runtime for Image3: {time1 - time_image3:0.0f} seconds")

Runtime so far: 8376 seconds
Runtime for Image3: 4375 seconds


### Verify which pipeline steps were run

In [32]:
# Identify *_i2d file and open as datamodel
i2d = glob.glob(os.path.join(image3_dir, "*_i2d.fits"))[0]
i2d_f = datamodels.open(i2d)

# Check which steps were run
i2d_f.meta.cal_step.instance

{'assign_wcs': 'COMPLETE',
 'charge_migration': 'COMPLETE',
 'dark_sub': 'COMPLETE',
 'dq_init': 'COMPLETE',
 'flat_field': 'COMPLETE',
 'gain_scale': 'SKIPPED',
 'group_scale': 'SKIPPED',
 'ipc': 'SKIPPED',
 'jump': 'COMPLETE',
 'linearity': 'COMPLETE',
 'outlier_detection': 'COMPLETE',
 'persistence': 'SKIPPED',
 'photom': 'COMPLETE',
 'ramp_fit': 'COMPLETE',
 'refpix': 'COMPLETE',
 'resample': 'COMPLETE',
 'saturation': 'COMPLETE',
 'skymatch': 'COMPLETE',
 'superbias': 'COMPLETE',
 'tweakreg': 'COMPLETE'}

Check which reference files were used to calibrate the dataset

In [33]:
i2d_f.meta.ref_file.instance

{'area': {'name': 'crds://jwst_niriss_area_0021.fits'},
 'camera': {'name': 'N/A'},
 'collimator': {'name': 'N/A'},
 'crds': {'context_used': 'jwst_1263.pmap', 'sw_version': '11.18.1'},
 'dark': {'name': 'crds://jwst_niriss_dark_0169.fits'},
 'dflat': {'name': 'N/A'},
 'disperser': {'name': 'N/A'},
 'distortion': {'name': 'crds://jwst_niriss_distortion_0047.asdf'},
 'drizpars': {'name': 'crds://jwst_niriss_drizpars_0002.fits'},
 'fflat': {'name': 'N/A'},
 'filteroffset': {'name': 'crds://jwst_niriss_filteroffset_0006.asdf'},
 'flat': {'name': 'crds://jwst_niriss_flat_0282.fits'},
 'fore': {'name': 'N/A'},
 'fpa': {'name': 'N/A'},
 'gain': {'name': 'crds://jwst_niriss_gain_0006.fits'},
 'ifufore': {'name': 'N/A'},
 'ifupost': {'name': 'N/A'},
 'ifuslicer': {'name': 'N/A'},
 'linearity': {'name': 'crds://jwst_niriss_linearity_0017.fits'},
 'mask': {'name': 'crds://jwst_niriss_mask_0017.fits'},
 'msa': {'name': 'N/A'},
 'ote': {'name': 'N/A'},
 'photom': {'name': 'crds://jwst_niriss_photo

## 8. Visualize the drizzle-combined image

We are using the [Imviz tool](https://jdaviz.readthedocs.io/en/latest/imviz/index.html) within the `jdaviz` package to visualize the NIRISS image.

In [39]:
# Create an Imviz instance and set up default viewer
imviz_i2d = Imviz()
viewer_i2d = imviz_i2d.default_viewer

# Read in the science array for our visualization dataset:
i2d_science = i2d_f.data

# Load the dataset into Imviz
imviz_i2d.load_data(i2d_science)

# Visualize the dataset:
imviz_i2d.show()

Application(config='imviz', docs_link='https://jdaviz.readthedocs.io/en/v3.10.3/imviz/index.html', events=['ca…

In [40]:
viewer_i2d.stretch = 'sqrt'
viewer_i2d.set_colormap('Viridis')
viewer_i2d.cuts = '95%'

## <a id='detections'>Visualize Detected Sources</a>
Using the source catalog created by the `IMAGE3` stage of the pipeline, mark the detected sources, using different markers for point sources and extended sources. The source catalog is saved in `image3/image3_association_cat.ecsv` file. We will need to read in the `i2d` file again to make sure the world coordinate system (WCS) info is read in.

### Read in catalog file and identify point/extended sources

In [41]:
catalog_file = glob.glob(os.path.join(image3_dir, "*_cat.ecsv"))[0]
catalog = Table.read(catalog_file)

# To identify point/extended sources, use the 'is_extended' column in the source catalog
pt_src, = np.where(~catalog['is_extended'])
ext_src, = np.where(catalog['is_extended'])

# Define coordinates of point and extended sources
pt_coord = Table({'coord': [SkyCoord(ra=catalog['sky_centroid'][pt_src].ra,
                                     dec=catalog['sky_centroid'][pt_src].dec)]})
ext_coord = Table({'coord': [SkyCoord(ra=catalog['sky_centroid'][ext_src].ra,
                                      dec=catalog['sky_centroid'][ext_src].dec)]})

### Mark the extended and point sources on the image

Display combined image:

In [42]:
# Read in i2d file to Imviz
imviz_cat = Imviz()
viewer_cat = imviz_cat.default_viewer
imviz_cat.load_data(i2d)

# Adjust settings for viewer
viewer_cat.stretch = 'sqrt'
viewer_cat.set_colormap('Viridis')
viewer_cat.cuts = '95%'

imviz_cat.show()

Application(config='imviz', docs_link='https://jdaviz.readthedocs.io/en/v3.10.3/imviz/index.html', events=['ca…

Point sources will be marked by small pink circles and extended sources will be marked by larger white circles.

In [43]:
# Add marker for point sources:
viewer_cat.marker = {'color': 'pink', 'markersize': 50, 'fill': False}

viewer_cat.add_markers(pt_coord, use_skycoord=True, marker_name='point_sources')

# Add marker for extended sources:
viewer_cat.marker = {'color': 'white', 'markersize': 100, 'fill': False}

viewer_cat.add_markers(ext_coord, use_skycoord=True, marker_name='extended_sources')